In [1]:
from base import generator_gellmann
from base import generator_haar

def init_experiment(n):
    d = 2**n

    # Generate 6^n density matrices
    rho_list = generator_haar.generate_n_qubits_rho_haar(n)
    print(f"Generated {len(rho_list)} of {rho_list[0].shape} rho.")

    # Generate epsilon
    epsilon = generator_haar.random_unitary(d)
    print(f"Generated {epsilon.shape} epsilon.")

    # Generate K list
    unitary = generator_haar.haar(d)
    kraus_operators = generator_haar.generate_kraus_operators(unitary)
    print(f"Generated {len(kraus_operators)} of {kraus_operators[0].shape} kraus operators.")
    return rho_list, epsilon, kraus_operators

2025-02-26 15:10:24.677370: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 15:10:24.677927: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 15:10:24.680637: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 15:10:24.688418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740550224.701840  348253 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740550224.70

In [2]:
from base import epsilon_rho
def calculate_rho2_lists(rho_list, epsilon, kraus_operators):
    rho2 = []
    rho2_kraus = []
    for rho in rho_list:
        rho2.append(epsilon_rho.calculate_from_unitary(rho, epsilon))
        rho2_kraus.append(epsilon_rho.calculate_from_kraus_operators(rho, kraus_operators))
    return rho2, rho2_kraus

In [3]:
import numpy as np
import tensorflow as tf
def write_to_file(filename, data):
    """Write TensorFlow tensor data to a text file without truncation."""
    tensor_data = data.numpy() if isinstance(data, tf.Tensor) else data

    # Open the file and write the tensor data
    with open(filename, 'w') as f:
        if isinstance(data, np.ndarray):
            np.savetxt(f, data, fmt="%.6f")
        elif isinstance(data, list):
            for item in data:
                f.write(f"{item}\n")
        else:
            f.write(str(data))

In [9]:
import os
from base import optimize_algorithm
from base import metrics
experiment_folder = 'results/experiment_new/haar_random'

for num_qubits in range(2, 4):
    if (experiment_folder == ''):
        break
    else:
        write_folder = os.path.join(experiment_folder, str(num_qubits) + "_qubits")
        if not os.path.exists(write_folder):
            os.makedirs(write_folder)
    print(f"N={num_qubits}")

    #-----Init experiment-----
    rho_list, epsilon, kraus_operators = init_experiment(num_qubits)

    #-----Learn kraus operators-----
    kraus_operators_res, cost_dict = optimize_algorithm.optimize_adam_kraus_set(rho_list, epsilon, kraus_operators, num_qubits, 0.1, num_loop=200)
    
    #-----Calculate result data-----
    rho3_list = epsilon_rho.calculate_set_from_kraus_operators(kraus_operators_res, rho_list, epsilon)
    rho2_list, rho2_kraus_list = calculate_rho2_lists(rho_list, epsilon, kraus_operators_res)
    
    mean_fidelity_rho_rho3 = metrics.mean_fidelity(rho3_list, rho_list)
    mean_fidelity_rho2_rho2 = metrics.mean_fidelity(rho2_kraus_list, rho2_list)

    #-----Write to folder-----
    write_to_file(os.path.join(write_folder, "rho_list.txt"), rho_list)
    write_to_file(os.path.join(write_folder,"epsilon.txt"), epsilon)
    
    write_to_file(os.path.join(write_folder,"kraus_operators.txt"), kraus_operators_res)
    write_to_file(os.path.join(write_folder,"cost_dict.txt"), cost_dict)

    write_to_file(os.path.join(write_folder,"rho3_list.txt"), rho3_list)
    write_to_file(os.path.join(write_folder,"rho2_list.txt"), rho2_list)
    write_to_file(os.path.join(write_folder,"rho2_kraus_list.txt"), rho2_kraus_list)

    write_to_file(os.path.join(write_folder,"mean_fidelity_rho_rho3.txt"), mean_fidelity_rho_rho3)
    write_to_file(os.path.join(write_folder,"mean_fidelity_rho2_rho2.txt"), mean_fidelity_rho2_rho2)

    
    

N=2
Generated 36 of (4, 4) rho.
Generated (4, 4) epsilon.
Generated 4 of (4, 4) kraus operators.
tf.Tensor(0.9646329841563025, shape=(), dtype=float64)


tf.Tensor(0.8344169136036174, shape=(), dtype=float64)
tf.Tensor(0.5727830759208371, shape=(), dtype=float64)
tf.Tensor(0.3444301507012484, shape=(), dtype=float64)
tf.Tensor(0.22658838099090778, shape=(), dtype=float64)
tf.Tensor(0.1552185320408375, shape=(), dtype=float64)
tf.Tensor(0.13745894627968983, shape=(), dtype=float64)
tf.Tensor(0.13815091116730627, shape=(), dtype=float64)
tf.Tensor(0.12232374784152888, shape=(), dtype=float64)
tf.Tensor(0.1055056383969756, shape=(), dtype=float64)
tf.Tensor(0.10533510072472482, shape=(), dtype=float64)
tf.Tensor(0.133703926439318, shape=(), dtype=float64)
tf.Tensor(0.11047864367092164, shape=(), dtype=float64)
tf.Tensor(0.08967612535674917, shape=(), dtype=float64)
tf.Tensor(0.08331464713321672, shape=(), dtype=float64)
tf.Tensor(0.06811340682902144, shape=(), dtype=float64)
tf.Tensor(0.05907965063850836, shape=(), dtype=float64)
tf.Tensor(0.053399496643939476, shape=(), dtype=float64)
tf.Tensor(0.049150643043559386, shape=(), dtype=float6